In [ ]:
import networkx as nximport itertools as itfrom scipy.optimize import minimizeimport matplotlib.pyplot as pltimport numpy as np

In [ ]:
def pairs(li):    return [(li[i], li[(i + 1) % len(li)]) for i in range(len(li))]

In [ ]:
g = nx.Graph()

In [ ]:
class NBQP:    def __init__(self, ingraph, rotation, coloring=None):        self.graph = ingraph        self.rotation = rotation        self.nnum = self.graph.number_of_nodes()        self.quads = self.compute_quads()        self.coloring = coloring        self.prevdata = None    def compute_quads(self):        quadslist = set()        for v, neig in self.rotation.items():            for w1, w2 in pairs(neig):                v2ind = (self.rotation[w1].index(v) + 1) % len(self.rotation[w1])                v2 = self.rotation[w1][v2ind]                newquad = [v, w1, v2, w2]                start = np.argmin(newquad)                if start % 2 == 0:                    ordered = tuple(newquad[start:] + newquad[:start])                    quadslist.add(ordered)        return quadslist    def diagonal_list(self):        return [(quad[0], quad[2]) for quad in self.quads] + [(quad[1], quad[3]) for quad in self.quads]    def diag_graph(self):        G = nx.Graph()        G.add_edges_from(self.diagonal_list())        return G    def __repr__(self):        return f"NBQP graph:{list(self.graph.edges)} rotation:{self.rotation} quads:{self.quads} cols:{self.coloring}"    def __str__(self):        return f"NBQP graph:{list(self.graph.edges)} rotation:{self.rotation} quads:{self.quads} cols:{self.coloring}"

In [ ]:
k4 = nx.Graph()k4.add_edges_from([(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)])q4rotation = {0: (1, 3, 2), 1: (0, 2, 3), 2: (0, 3, 1), 3: (0, 1, 2)}k4coloring = {0: 0, 1: 1, 2: 2, 3: 3}q4 = NBQP(k4, q4rotation, k4coloring)print(q4)

In [ ]:
q4.diagonal_list()

In [ ]:
def split(q, a, b1, b2):    newgraph = q.graph.copy()    rotation = q.rotation    rota = rotation[a]    i1 = rota.index(b1)    i2 = rota.index(b2)    if i2 == min(i1, i2):        b1, b2 = b2, b1    i1, i2 = min(i1, i2), max(i1, i2)    n1 = rota[i1 + 1:i2]    n2 = rota[i2 + 1:] + rota[:i1]    newrot = rotation.copy()    del newrot[a]    if 0 not in newrot.keys():        newind1 = 0    else:        newind1 = min(i + 1 for i in newrot.keys() if i + 1 not in newrot.keys())    newrot[newind1] = ()    newind2 = min(i + 1 for i in newrot.keys() if i + 1 not in newrot.keys())    newgraph.remove_node(a)    newgraph.add_edges_from([(newind1, x) for x in n1])    newgraph.add_edges_from([(newind2, x) for x in n2])    newgraph.add_edges_from([(newind1, b1), (newind1, b2)])    newgraph.add_edges_from([(newind2, b1), (newind2, b2)])    newrot[newind1] = tuple([b1] + list(n1) + [b2])    newrot[newind2] = tuple([b2] + list(n2) + [b1])    for v in n1:        newrot[v] = tuple([newind1 if x == a else x for x in rotation[v]])    for v in n2:        newrot[v] = tuple([newind2 if x == a else x for x in rotation[v]])    newcoloring = None    if q.coloring:        newcoloring = {}        for v in newgraph.nodes:            if v == newind1 or v == newind2:                newcoloring[v] = q.coloring[a]            else:                newcoloring[v] = q.coloring[v]    return NBQP(newgraph, newrot, newcoloring)

In [ ]:
print(split(q4, 0, 1, 3))

In [ ]:
collection = {}collection[4] = {'nbqp_list': [q4], 'graph_list': [q4.graph]}

In [ ]:
collection[5] = {'nbqp_list': [], 'graph_list': []}for nbqp in collection[4]['nbqp_list']:    g = nbqp.graph    rot = nbqp.rotation    for v in list(g.nodes):        for w1, w2 in it.combinations(rot[v], 2):            newnbqp = split(nbqp, v, w1, w2)            newnbqp.prevdata = (nbqp, v, w1, w2)            found = False            for gr in collection[5]['graph_list']:                if nx.is_isomorphic(newnbqp.graph, gr):                    found = True                    break            if not found:                collection[5]['nbqp_list'].append(newnbqp)                collection[5]['graph_list'].append(newnbqp.graph)

In [ ]:
for j in [6, 7, 8, 9, 10]:    size = j    collection[size] = {'nbqp_list': [], 'graph_list': []}    for nbqp in collection[size - 1]['nbqp_list']:        g = nbqp.graph        rot = nbqp.rotation        for v in list(g.nodes):            for w1, w2 in it.combinations(rot[v], 2):                newnbqp = split(nbqp, v, w1, w2)                newnbqp.prevdata = (nbqp, v, w1, w2)                found = False                for gr in collection[size]['graph_list']:                    if nx.is_isomorphic(newnbqp.graph, gr):                        found = True                        break                if not found:                    collection[size]['nbqp_list'].append(newnbqp)                    collection[size]['graph_list'].append(newnbqp.graph)

In [ ]:
for j in [6, 7, 8, 9, 10, 11]:    size = j    collection[size] = {'nbqp_list': [], 'graph_list': []}    graph_hashes = set()    for nbqp in collection[size - 1]['nbqp_list']:        g = nbqp.graph        rot = nbqp.rotation        for v in list(g.nodes):            for w1, w2 in it.combinations(rot[v], 2):                newnbqp = split(nbqp, v, w1, w2)                newnbqp.prevdata = (nbqp, v, w1, w2)                gr = newnbqp.graph                gr_hash = nx.weisfeiler_lehman_graph_hash(gr)                if gr_hash not in graph_hashes:                    collection[size]['nbqp_list'].append(newnbqp)                    collection[size]['graph_list'].append(gr)                    graph_hashes.add(gr_hash)

In [ ]:
for i in [4, 5, 6, 7, 8, 9, 10, 11]:    print(i, len(collection[i]['graph_list']))

In [ ]:
from pysat.formula import IDPoolfrom pysat.solvers import Solverdef check_coloring_orientation(graph, k):    vpool = IDPool()    def col(v, c):        return vpool.id(("col", v, c))    def orient(u, v):        return vpool.id(("orient", u, v))    with Solver(name="glucose3") as solver:        for v in graph:            solver.add_clause([col(v, c) for c in range(k)])            for c1 in range(k):                for c2 in range(c1 + 1, k):                    solver.add_clause([-col(v, c1), -col(v, c2)])        for u in graph:            for v in graph[u]:                if u < v:                    for c in range(k):                        solver.add_clause([-col(u, c), -col(v, c)])        for u in graph:            for v in graph[u]:                if u < v:                    solver.add_clause([orient(u, v), orient(v, u)])                    solver.add_clause([-orient(u, v), -orient(v, u)])        for u in graph:            for v in graph[u]:                for w in graph[u]:                    if v < w:                        for c1 in range(k):                            for c2 in range(k):                                if c1 != c2:                                    solver.add_clause([-orient(u, v), -orient(u, w), -col(v, c1), -col(w, c2)])        sat = solver.solve()        if not sat:            return None, None        model = solver.get_model()        coloring = {}        orientation = {}        for v in graph:            for c in range(k):                if col(v, c) in model and model[model.index(col(v, c))] > 0:                    coloring[v] = c        for u in graph:            for v in graph[u]:                if orient(u, v) in model and model[model.index(orient(u, v))] > 0:                    orientation[(u, v)] = True                elif orient(v, u) in model and model[model.index(orient(v, u))] > 0:                    orientation[(u, v)] = False        return coloring, orientation

In [ ]:
colnum = 6for i in [4, 5, 6, 7, 8, 9, 10, 11]:    for g in range(len(collection[i]['graph_list'])):        gr = collection[i]['graph_list'][g]        adj = {u: list(gr.neighbors(u)) for u in gr.nodes}        coloring, orientation = check_coloring_orientation(adj, colnum)        if coloring and orientation:            print(f"Graph {g} of size {i} is colorable with {colnum} colors")            pos = nx.spring_layout(gr)            node_colors = [coloring[v] for v in gr.nodes]            nx.draw(gr, pos, with_labels=True, node_color=node_colors, cmap=plt.cm.Set1)            for (u, v) in gr.edges:                if orientation.get((u, v), False):                    plt.arrow(pos[u][0], pos[u][1], pos[v][0] - pos[u][0], pos[v][1] - pos[u][1],                              length_includes_head=True, head_width=0.05, color='black', alpha=0.7)                elif orientation.get((v, u), False):                    plt.arrow(pos[v][0], pos[v][1], pos[u][0] - pos[v][0], pos[u][1] - pos[v][1],                              length_includes_head=True, head_width=0.05, color='black', alpha=0.7)            plt.show()